# Correcting the map counts to account for diving seals and higher on-the-ground counts
This file includes the code that inflates the map counts using detection rate-estimating functions fitted with data from Erebus Bay, where ground counts could be conducted.  
  
In a nutshell, we use these models to estimate the detection rate, and then inflate the map counts according to this rate.

### Loading libraries, functions and data

In [22]:
libs<-c("ggplot2","plyr","dplyr")
lapply(libs, require, character.only = TRUE)
pathToLocalGit<-"/home/ubuntu/Workspace/ContinentalWESEestimates/"

source(paste0(pathToLocalGit,"scripts/countSealsFromTags_functions.R"))

## Load the data - the data were generated from notebook: "Count Seals From Tags.ipynb"
load(file=paste0(pathToLocalGit,"data/estimatesByMap_unadjusted.RData"))
head(df)

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

,regionMapId,RShour,overlayId,region,mapId,year,satId,numTaggers,estNumSeals,uclNumSeals,⋯,totalTags,avgTags,scaledTotalTags,scaledAvgTags,numHour,sinH,resid,estimateH,lowerH,upperH
,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AMU101663,21,60297,AMU,101663,2010,WV01,1,1,4,⋯,1,1,-0.27006566,-0.18956660,0.25,1,-0.4112584,1,1,4
2,AMU102072,21,60297,AMU,102072,2010,WV01,2,1,4,⋯,2,1,-0.24135869,-0.18956660,0.25,1,-0.3819680,1,1,4
3,AMU102099,21,60297,AMU,102099,2010,WV01,1,3,7,⋯,2,2,-0.24135869,-0.13233711,0.25,1,0.9205844,3,1,7
4,AMU103615,21,60297,AMU,103615,2010,WV01,1,3,7,⋯,3,3,-0.21265171,-0.07510762,0.25,1,0.2577237,3,1,7
5,AMU103695,21,60297,AMU,103695,2010,WV01,1,1,4,⋯,1,1,-0.27006566,-0.18956660,0.25,1,-0.4101991,1,1,4
6,AMU103791,21,60297,AMU,103791,2010,WV01,1,8,18,⋯,10,10,-0.01170287,0.32549884,0.25,1,0.6028688,8,1,18


Now we use the handy function to request the detection rates under both models. Note that we can specify the "weight" of island counts. This is the proportion of map locations that resemble the islet haul-out locations in Erebus Bay (i.e., Turk's Head-Tryggve and Hutton Cliffs). According to Michelle and David, the vast majority of counts came from such locations. Here we are conservative and assume 95% of locations are islets.

In [23]:
adjRates<-predictDetRates(pathToGit=pathToLocalGit,dat=df,keyFieldName="regionMapId",islandWeight=0.95)
head(adjRates)

,regionMapId,wgtPredColRate,wgtPredIslRate
,<chr>,<dbl>,<dbl>
1,AMU101663,0.3104196,0.2499636
2,AMU102072,0.3104196,0.2499636
3,AMU102099,0.3153436,0.2535324
4,AMU103615,0.3202675,0.2571012
5,AMU103695,0.3104196,0.2499636
6,AMU103791,0.3547351,0.2820827


Finally, we add the detection rates to our data.frame of map counts, inflate, and summarize the results.  
  
Since detectionRate = count-in-map/count-on-the-ground, and we want count-on-the-ground, then:
Count-on-the-ground = count-in-map/detectionRate

In [27]:
countdf<-merge(df,adjRates,by="regionMapId")

countdf$mdlColEstimate<-round(countdf$estimateH/countdf$wgtPredColRate)
countdf$mdlColUpper<-round(countdf$upperH/countdf$wgtPredColRate)
countdf$mdlColLower<-round(countdf$lowerH/countdf$wgtPredColRate)

countdf$mdlIslEstimate<-round(countdf$estimateH/countdf$wgtPredIslRate)
countdf$mdlIslUpper<-round(countdf$upperH/countdf$wgtPredIslRate)
countdf$mdlIslLower<-round(countdf$lowerH/countdf$wgtPredIslRate)

## Using the colony model:
print("Colony model")
estByRegionCol<-as.data.frame(countdf[,c("region","mdlColLower","mdlColEstimate","mdlColUpper")] %>% group_by(region) %>% dplyr::summarize(Lower=round(sum(mdlColLower)),Estimate=round(sum(mdlColEstimate)),Upper=round(sum(mdlColUpper))))
estByRegionCol<-rbind(estByRegionCol,data.frame(region="Total",Lower=round(sum(countdf$mdlColLower)),Estimate=round(sum(countdf$mdlColEstimate)),Upper=round(sum(countdf$mdlColUpper))))
print(estByRegionCol)

[1] "Colony model"
  region Lower Estimate  Upper
1    AMU  3163     9264  23657
2    EA1  7225    22198  56085
3    EA2  6541    19199  48854
4    QMA  8253    21054  55504
5    RSS 24886    74663 192329
6    WAP  8530    21644  57132
7  Total 58598   168022 433561


In [28]:
## Using the islet/mainland model:
print("Islet/mainland model")
estByRegionIsl<-as.data.frame(countdf[,c("region","mdlIslLower","mdlIslEstimate","mdlIslUpper")] %>% group_by(region) %>% dplyr::summarize(Lower=round(sum(mdlIslLower)),Estimate=round(sum(mdlIslEstimate)),Upper=round(sum(mdlIslUpper))))
estByRegionIsl<-rbind(estByRegionIsl,data.frame(region="Total",Lower=round(sum(countdf$mdlIslLower)),Estimate=round(sum(countdf$mdlIslEstimate)),Upper=round(sum(countdf$mdlIslUpper))))
print(estByRegionIsl)

[1] "Islet/mainland model"
  region Lower Estimate  Upper
1    AMU  4185    11725  29601
2    EA1  9555    28095  70342
3    EA2  8681    24289  61075
4    QMA 10903    26768  69478
5    RSS 32837    94707 241425
6    WAP 11265    27504  71481
7  Total 77426   213088 543402
